AI SHIFTの記事を参考にしたもの

In [5]:
!pip install bert-serving-client

In [3]:
!pip install sentencepiece

In [1]:
from bert_serving.client import BertClient
import sentencepiece as spm

In [2]:
class BertServingClient:
    def __init__(self, sp_model='./bertserving/model/wiki-ja.model', bert_client_ip='0.0.0.0'):
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(sp_model)
        self.bc = BertClient(ip=bert_client_ip)
        
    def sentence_piece_tokenizer(self, text):
        text = text.lower()
        return self.sp.EncodeAsPieces(text)
    
    def sentence2vec(self, sentences):
        parsed_texts = list(map(self.sentence_piece_tokenizer, sentences))
        return self.bc.encode(parsed_texts, is_tokenized=True)

In [ ]:
bsc = BertServingClient()

In [ ]:
sentences = ['今日は晴れです', '明日は雨です']
vectors = bsc.sentence2vec(sentences)
print(vectors)

## ここからvert抜き

In [99]:
from elasticsearch import Elasticsearch

index = 'index9'
index_file = './index.json'

es = Elasticsearch(['3.23.190.53'], port=9200, use_ssl=False, verify_certs=False)
with open(index_file) as f:
    source = f.read().strip()
    print(es.indices.create(index, source))

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'index9'}


In [103]:
import pandas as pd
from collections import OrderedDict
from elasticsearch.helpers import bulk

In [104]:
def search_with_kuromoji(query, es, index):

    response = es.search(
        index=index,
        body={
            "query": {
                "match": {
                    "text_kuromoji": query
                }
            }
        }
    )
    
    return pd.DataFrame([
        OrderedDict({
            'text_kuromoji': row['_source']['text_kuromoji'], 
            'score': row['_score']
        }) for _, row in pd.DataFrame(response['hits']['hits']).iterrows()])

In [108]:
texts = [
    '今日は晴れです',
    '明日は雨です',
    '今日は暑いです',
    '明日は涼しいです'
]

docs = [
    {
        'text_kuromoji': text,  
        '_index': index
    } 
    for text in zip(texts)
]

bulk(es, docs)

(4, [])

In [109]:
query = '今日は晴れです'
search_with_kuromoji(query,es,index)

,text_kuromoji,score
0,[今日は晴れです],1.414214
1,[今日は暑いです],0.707107


In [110]:
query = '今日は雨です'
search_with_kuromoji(query,es,index)

,text_kuromoji,score
0,[今日は晴れです],0.707107
1,[明日は雨です],0.707107
2,[今日は暑いです],0.707107
